# 0. Импорты

In [ ]:
import pandas as pd
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.metrics import r2_score, classification_report, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
import numpy as np
import plotly.io as pio
import plotly.express as px

pio.renderers.default = "colab"

/home/sebulba/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def plot_weights(coefs, columns=None, show_cols=False):
  df_plot = pd.DataFrame()

  df_plot['y'] = coefs.squeeze()
  if show_cols:
    df_plot['x'] = columns
  else:
    df_plot['x'] = ['w' + str(s) for s in range(len(coefs.squeeze()))]
  df_plot['c'] = df_plot.y.apply(lambda x: '>=0' if x >= 0 else '<0')


  return px.bar(df_plot, y='y', x='x', color='c', color_discrete_map={'>=0': 'blue', '<0': 'red'}).update_xaxes(categoryorder='array', categoryarray=df_plot['x'])

# ОПТИМИЗАЦИЯ ЛИНЕЙНОЙ РЕГРЕССИИ

## 1.  Для набора данных Cars Moldova

Загрузим датасет

In [ ]:
df = pd.read_csv('cars_moldova_feature.csv').drop(columns='Unnamed: 0')
df['Year'] = pd.to_datetime(df['Year'].apply(lambda x: str(x) + '-01-01 00:00:01'))
df['Year'] = df.Year.values.astype(np.int64) // 10 ** 9
df

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),year_distance,distance_type,Make_rarity,Price_cat,Year_category
0,Toyota,Prius,1293840001,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0,17727.272727,high,rare,cheap,regular
1,Renault,Grand Scenic,1388534401,Universal,135000.0,1500.0,Diesel,Manual,8550.0,16875.000000,average,rare,cheap,regular
2,Volkswagen,Golf,883612801,Hatchback,1.0,1400.0,Petrol,Manual,2200.0,0.041667,optimal,rare,average,old
3,Renault,Laguna,1325376001,Universal,110000.0,1500.0,Diesel,Manual,6550.0,11000.000000,average,rare,cheap,regular
4,Opel,Astra,1136073601,Universal,200000.0,1600.0,Metan/Propan,Manual,4100.0,12500.000000,high,rare,average,old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36019,Land Rover,Freelander,1009843201,Crossover,225000.0,1800.0,Metan/Propan,Manual,4400.0,11250.000000,high,common,average,old
36020,Dacia,Logan Mcv,1420070401,Universal,89000.0,1500.0,Diesel,Manual,7000.0,12714.285714,optimal,rare,cheap,regular
36021,Renault,Modus,1230768001,Hatchback,225.0,1500.0,Diesel,Manual,4500.0,17.307692,optimal,rare,average,old
36022,Mazda,6,1136073601,Combi,370000.0,2000.0,Diesel,Manual,4000.0,23125.000000,very_high,rare,average,old


Сделали даты таймштампами, теперь они более полезны для модели

In [ ]:
X = df.drop(columns='Price(euro)')
y = df['Price(euro)']

In [ ]:
numeric_features = ['Distance', 'Engine_capacity(cm3)', 'year_distance', 'Year']
numeric_transformer = Pipeline(
    steps=[('poly', PolynomialFeatures(degree=2, include_bias=False)),  ("pca", PCA(n_components=12)), ("scaler", MinMaxScaler())]
)

categorical_features = ['Make', 'Model', 'Style', 'Fuel_type', 'Transmission',
                        'distance_type', 'Make_rarity', 'Price_cat', 'Year_category']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)   
    ]
)

Итак, что мы зедсь сделали, идёт в первую очередь создание полиномиальных фичей, а потом уже метод главных компонент, который не особо таки и нужен, так как модель получается не особо сложная, но мы убрали две компоненты, снизив точность всего на пол процента где-то.

А дальше просто создание категориальных фич

Опытным путём было выявлено, что модель сильно ошибалась, когда мы ей добавили больше полиномиальных фич, которым она давала большие веса, и метрика R2 выходила 0.62, а также когда снижали размерность ниже, чем стоит сейчас.

In [ ]:
reg = Pipeline(
    steps=[("preprocessor", preprocessor), ("ridge", RidgeCV(alphas=(0.1, 1, 3, 6, 10, 15)))]
)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

reg.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('poly',
                                                                   PolynomialFeatures(include_bias=False)),
                                                                  ('pca',
                                                                   PCA(n_components=12)),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['Distance',
                                                   'Engine_capacity(cm3)',
                                                   'year_distance', 'Year']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Make', 'Model', 'Style',
                                                   'Fuel_type', 'Transmission',
                                                   'distance_type',
                                                   'Make_rarity', 'Price_cat',
                                                   'Year_category'])])),
                ('ridge', RidgeCV(alphas=(0.1, 1, 3, 6, 10, 15)))])

На диаграмме прекрасно выдно, как работает пайплайн

In [ ]:
y_pred = reg.predict(X_test)

print(f'R2 score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test, y_pred, squared=False)}')
print(f'MAPE: {mean_absolute_percentage_error(y_test, y_pred)}')

R2 score: 0.8064589569950403
MAE: 2124.8565267430354
RMSE: 3413.4387251511134
MAPE: 0.4140478693568304


Видим, что точность вырасла на 6% по сравнению с прошлыми попытками предсказать цену автомобиля, это прекрасный результат

Остальные показатели также снизились

In [ ]:
plot_weights(reg['ridge'].coef_)

Из-за метода главных компонент, у нас теперь нет названий числовых признаков, которые расположены в начале, но мы понимаем, что это они находятся в начале и ridge регрессия в таком формате большие веса даёт именно числовым характеристикам автомобилей

# ОПТИМИЗАЦИЯ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ

## 1.  Для набора данных Cars Moldova

Возьмём тот же датасет

In [ ]:
df

,Make,Model,Year,Style,Distance,Engine_capacity(cm3),Fuel_type,Transmission,Price(euro),year_distance,distance_type,Make_rarity,Price_cat,Year_category
0,Toyota,Prius,1293840001,Hatchback,195000.0,1800.0,Hybrid,Automatic,7750.0,17727.272727,high,rare,cheap,regular
1,Renault,Grand Scenic,1388534401,Universal,135000.0,1500.0,Diesel,Manual,8550.0,16875.000000,average,rare,cheap,regular
2,Volkswagen,Golf,883612801,Hatchback,1.0,1400.0,Petrol,Manual,2200.0,0.041667,optimal,rare,average,old
3,Renault,Laguna,1325376001,Universal,110000.0,1500.0,Diesel,Manual,6550.0,11000.000000,average,rare,cheap,regular
4,Opel,Astra,1136073601,Universal,200000.0,1600.0,Metan/Propan,Manual,4100.0,12500.000000,high,rare,average,old
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36019,Land Rover,Freelander,1009843201,Crossover,225000.0,1800.0,Metan/Propan,Manual,4400.0,11250.000000,high,common,average,old
36020,Dacia,Logan Mcv,1420070401,Universal,89000.0,1500.0,Diesel,Manual,7000.0,12714.285714,optimal,rare,cheap,regular
36021,Renault,Modus,1230768001,Hatchback,225.0,1500.0,Diesel,Manual,4500.0,17.307692,optimal,rare,average,old
36022,Mazda,6,1136073601,Combi,370000.0,2000.0,Diesel,Manual,4000.0,23125.000000,very_high,rare,average,old


In [ ]:
X = df.drop(columns='Transmission')
y = df['Transmission']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
numeric_features = ['Distance', 'Engine_capacity(cm3)', 'year_distance', 'Year', 'Price(euro)']
numeric_transformer = Pipeline(
    steps=[('poly', PolynomialFeatures(degree=3, include_bias=False)),  ("pca", PCA(n_components=13)), ("scaler", StandardScaler())]
)

categorical_features = ['Make', 'Model', 'Style', 'Fuel_type', 
                        'distance_type', 'Make_rarity', 'Price_cat', 'Year_category']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")


preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)   
    ]
)

Модель также ошибалась при большом количестве полиномиальных фич и при нормализации, на автоматических коробках

Были попробованы такие параметры:
- `penalty` = l1:
  * `solver` = liblinear
  * `max_iter` = 100
  * `tol` = 0.001
  * `cv` = 5

- `penalty` = l2:
  * `solver` = liblinear, lbfgs, newton-cg
  * `max_iter` = 100, 1000
  * `tol` = 0.001, 0.0001
  * `cv` = 5


In [ ]:
log_reg = Pipeline(
    steps=[("preprocessor", preprocessor), ("log", LogisticRegressionCV(penalty='l2', tol=0.0001, max_iter=1000, solver='newton-cg', cv=5))]
)

log_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('poly',
                                                                   PolynomialFeatures(degree=3,
                                                                                      include_bias=False)),
                                                                  ('pca',
                                                                   PCA(n_components=13)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Distance',
                                                   'Engine_capacity(cm3)',
                                                   'year_distance', 'Year',
                                                   'Price(euro)']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Make', 'Model', 'Style',
                                                   'Fuel_type', 'distance_type',
                                                   'Make_rarity', 'Price_cat',
                                                   'Year_category'])])),
                ('log',
                 LogisticRegressionCV(cv=5, max_iter=1000,
                                      solver='newton-cg'))])

In [ ]:
y_pred = log_reg.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   Automatic       0.88      0.80      0.83      3162
      Manual       0.85      0.91      0.88      4043

    accuracy                           0.86      7205
   macro avg       0.86      0.85      0.86      7205
weighted avg       0.86      0.86      0.86      7205



Результат, почему-то не меняется, по сравнению с тем, что было при прошлых попытках предсказать тип коробки передач, однако, в данном случае использование пайплайна отразилось на времени затраченном на построение прогноза и валидацию модели.

In [ ]:
plot_weights(log_reg['log'].coef_)

Ну и как видим, большее предпочтение отдано моделям машин, которые составляют большниство фич, и наши новые фичи вообще практически веса не получили, они находятся в самом конце

# Вывод

- В целом регрессия получилась замечательно, из-за удобства pipeline удалось с лёгсотью использовать и полиномиальные фичи и снижение размерности, посредством чего увеличить точность на целых 6% с уменьшением сложности модели

- Логистическую регрессию сложно как-то хорошо настроить, потому что вероятно она работает только на моделях автомобилей и даже не даёт ввеса новым признакам, следовательно предобработка не сделает ничего лучше, только настойка самой модели, которая ничего не дала по итогу.